## Access to raw files

In [7]:
import pandas as pd
import os

files_code_APE=os.path.abspath("INSEE_code_APE.ipynb")
dossier_notebook = os.path.dirname(files_code_APE)
dossier_project = os.path.dirname(dossier_notebook)
dossier_data=os.path.join(dossier_project,"data")
files_label=os.path.join(dossier_data,"int_courts_naf_rev_2.xls")
files_niv=os.path.join(dossier_data,"naf2008_5_niveaux.xls")

# To import the 2 documents of INSEE; one with the APE Codewith different Level (5 Levels) 
# and the other document for the signification of each APE Code
liste_niv=pd.read_excel(files_niv,dtype=str)
liste_niv_avec_int=pd.read_excel(files_label)


## Data cleaning

In [3]:
# To remove lines we dont need
liste_niv_avec_int=liste_niv_avec_int.dropna(axis=0)

# To keep columns we need
liste_niv_avec_int_sub=liste_niv_avec_int[[" Intitulés de la  NAF rév. 2, version finale ","Code"]]

# To change the name to something more simple
liste_niv_avec_int_sub=liste_niv_avec_int_sub.rename(columns={" Intitulés de la  NAF rév. 2, version finale ":"Label"})

# To remove the word SECTION because we only get Code with letter
# For exemple we dont want the "SECTION A" we just need the "A" for the APE Code
liste_niv_avec_int_sub=liste_niv_avec_int_sub.map(lambda x: x[-1:] if "SECTION" in x else x)



## files Merging 

In [4]:
# Create a new table with the 2 others with a merge on the APE Code NIV5 (NIV5=Level5)
# we also change the name of the Label into Label NIV5 because its the Label only for the NIV5 (Level5)
Merge_niv5=pd.merge(liste_niv,liste_niv_avec_int_sub,how="left",left_on="NIV5",right_on="Code").rename(columns={"Label":"Label NIV5"})

# delete the Code Columns of the second document because we already got APE Code on the first document 
del Merge_niv5["Code"]

# We repeat that for the APE Code Level 4 / 3 / 2 / 1
Merge_niv4=pd.merge(Merge_niv5,liste_niv_avec_int_sub[["Label","Code"]],how="left",left_on="NIV4",right_on="Code").rename(columns={"Label":"Label NIV4"})
del Merge_niv4["Code"]
Merge_niv3=pd.merge(Merge_niv4,liste_niv_avec_int_sub[["Label","Code"]],how="left",left_on="NIV3",right_on="Code").rename(columns={"Label":"Label NIV3"})
del Merge_niv3["Code"]
Merge_niv2=pd.merge(Merge_niv3,liste_niv_avec_int_sub[["Label","Code"]],how="left",left_on="NIV2",right_on="Code").rename(columns={"Label":"Label NIV2"})
del Merge_niv2["Code"]
Merge_niv1=pd.merge(Merge_niv2,liste_niv_avec_int_sub[["Label","Code"]],how="left",left_on="NIV1",right_on="Code").rename(columns={"Label":"Label NIV1"})
del Merge_niv1["Code"]

# set the index of the table with the APE Code of NIV5 (Level5)
Merge_niv1=Merge_niv1.set_index("NIV5")


## APE Code Function 

In [5]:
# Function to find the Label of an APE Code with the choice of wich Level of Label
def Code_APE_Label(Code,NIV):
    NIV="Label NIV"+str(NIV)
    return(Merge_niv1.loc[Code,NIV])
Code_APE_Label("99.00Z",1)

'ACTIVITÉS EXTRA-TERRITORIALES'

## Saving the new file

In [6]:
# Save the table as "APE_Code.xlsx"
Merge_niv1.to_excel("APE_Code.xlsx")